In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean

from avalanche.models import IcarlNet
from avalanche.training.supervised import ICaRL
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [3]:
pixel_transforms = transforms.Compose([transforms.ToTensor()])
per_pixel_mean = get_dataset_per_pixel_mean(CIFAR10('../data', train=True, download=True, transform=pixel_transforms))

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - per_pixel_mean,
              icarl_cifar100_augment_data,
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - per_pixel_mean,
              ]
       ),
       None,
       )
)

train_set = CIFAR10('../data', train=True, download=True)
test_set = CIFAR10('../data', train=False, download=True)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

170499072it [12:31, 226834.56it/s]                               


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# train_transform=transforms.Compose([transforms.ToTensor(),
#                           lambda img_pattern: img_pattern - per_pixel_mean,
#                           icarl_cifar100_augment_data])

# eval_transform=transforms.Compose([transforms.ToTensor(),
#                           lambda img_pattern: img_pattern - per_pixel_mean])

# splitcifar = SplitCIFAR10(n_experiences=5, seed=seed, train_transform=train_transform, eval_transform=eval_transform, dataset_root='../data')

# train_set = splitcifar.train_stream
# test_set = splitcifar.test_stream

# print(splitcifar.classes_order)

In [6]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger("iCaRL-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


KeyboardInterrupt: Interrupted by user

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 5
lr_milestones = [49, 63]
lr_factor = 5.0
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order,
                        )

model = IcarlNet(num_classes=num_class)    # n = ResidualBlock, c = input_dim
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)
sched = LRSchedulerPlugin(
        MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor)
    )

In [ ]:
memory_size = 2000
train_batch = 64
eval_batch = 32
epoch = 70

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

pllugins :  [<avalanche.training.plugins.lr_scheduling.LRSchedulerPlugin object at 0x000001FF26150EE0>, <avalanche.training.strategies.icarl._ICaRLPlugin object at 0x000001FF260FC130>, <avalanche.training.losses.ICaRLLossPlugin object at 0x000001FF2457DE20>, <avalanche.training.plugins.evaluation.EvaluationPlugin object at 0x000001FF260FC1F0>]


In [ ]:

for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 157/157 [00:07<00:00, 20.86it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5032
100%|██████████| 157/157 [00:04<00:00, 32.10it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:05<00:00, 31.32it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:05<00:00, 31.14it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:04<00:00, 31.60it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:04<00:00, 31.87it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:04<00:00, 32.23it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 157/157 [00:04<00:0